# Load and query data: `db.load & db.query`

In [ ]:
import lamindb as ln

ln.nb.header()

## Load

Load data objects into memory via {class}`~lamindb.db.load`:

In [ ]:
dobject = ln.db.query.dobject(name="iris").first()

df = ln.db.load(dobject)

In [ ]:
df.head()

To trace data back to its source, you can use the linked `dtransform` that generated the data. 

In [ ]:
dobject.dtransform_id

## Query

For simple frequent queries, LaminDB has a high-level query API {class}`~lamindb.db.query` based on [SQLModel](https://sqlmodel.tiangolo.com).

For complicated queries, you can use any SQL query. See below.

### Simple table queries

You can query any entity table in the database by its fields, for instance:

In [ ]:
ln.db.query.dobject(suffix=".feather").all()

The high-level query API by default returns a list of results:

In [ ]:
ln.db.query.species(taxon_id=10090).all()

You can call `.df()` to return a `DataFrame` instead.

In [ ]:
ln.db.query.species(taxon_id=10090).df()

If no constraints are passed, the query returns all rows in the table.

In [ ]:
ln.db.query.readout().all()

### Query data objects by linked entities

You can query data objects by fields that are not present in the `dobject` table via linked entities.

You can do this through providing a `where` dictionary.

In [ ]:
ln.db.query.dobject(suffix=".h5ad", where={"gene": {"symbol": "Actg1"}}).all()

`where` can filter conditions from multiple entities.

In [ ]:
ln.db.query.dobject(
    where={"gene": {"ncbi_gene_id": 66722}, "readout": {"efo_id": "EFO:0008913"}}
).all()

### SQL queries

You can construct arbitrary SQL queries via [SQLModel](https://sqlmodel.tiangolo.com).

For instance, let's query for a data source:

In [ ]:
with ln.db.session() as session:
    dtransform = session.get(ln.schema.core.dtransform, dobject.dtransform_id)

Inspecting the result we see that the dobject originates from a Jupyter Notebook.

In [ ]:
dtransform

In [ ]:
with ln.db.session() as session:
    jupynb = session.get(
        ln.schema.core.jupynb,
        (dtransform.jupynb_id, dtransform.jupynb_v),  # it's version "1" see jupynb_v
    )

In [ ]:
jupynb

Now we found the person who last edited the notebook!

In [ ]:
with ln.db.session() as session:
    user = session.get(ln.schema.core.user, jupynb.user_id)

In [ ]:
user